<a href="https://colab.research.google.com/github/jinseriouspark/data_analysis_with_llm/blob/main/colab_t4_gpu_running_for_ollma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- colab으로 사용할 수 있는 gpu를 local 또는 다른 노트북에서 사용하기 위해 켜두는 코드입니다.
- [ngrok](https://ngrok.com/)에 계정을 생성하고, 무료 버전을 이용할 수 있는 토큰을 받아 cred.txt로 저장하고 실행하였습니다.
- 마지막 코드 실행 시, 출력물 중 url 주소를 따로 저장해주셔야 합니다.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && suedo apt-get install -y cuda-drivers
!pip install pyngrok


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
from pyngrok import ngrok

In [3]:
with open('/content/drive/MyDrive/polaris-llm/cred.txt', 'r') as f:
  token = [r.strip() for r in f.readlines()]

In [4]:
ngrok.set_auth_token(token[0])

NameError: name 'ngrok' is not defined

In [13]:
import os
import asyncio

# set LD_LIBRARY_PATH
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

In [ ]:
async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )
from IPython.display import clear_output
clear_output()


await asyncio.gather(
  run_process(['ollama', 'serve']),
  run_process(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header="localhost:11434"']),
  run_process(['ollama', 'pull', 'gemma:7b-instruct']),
  )
